# Driver Code

In [0]:
# notebook_path = "/Workspace/Repos/prem@loyalytics.in/Private-Repo/Assortment Optimization/Assortment Optimization 2.0/Assortment Optimization Framework 2.0"


# categories = ['PASTA & NOODLE', 'PASTA & NOODLE', 'PASTA & NOODLE', 'COOKING OILS & GHEE', 'COOKING OILS & GHEE', 'COOKING OILS & GHEE', 'COOKING OILS & GHEE', 'ICE CREAM & DESSERTS']

# material_groups = ['PASTA', 'INSTANT NOODLE', 'CUP NOODLE', 'COCONUT OIL', 'OLIVE OIL', 'SUNFLOWER OIL', 'VEGETABLE OIL', 'FRUIT JUICES']

# current_dates = ['2024-03-01', '2024-03-01', '2024-03-01', '2024-05-01', '2024-06-01', '2024-06-01', '2024-06-01', '2024-07-01']

# regions = ['ABU DHABI', 'AL AIN', 'DUBAI', 'SHARJAH']

# for i in range(len(categories)):
#     for region in regions:
#         result = dbutils.notebook.run(notebook_path, timeout_seconds = 0, arguments = {
#             "Category": categories[i],
#             "Material Group": material_groups[i],
#             "Region": region,
#             "Current Date": current_dates[i]
#         })

#         print(f"Run Completed - {material_groups[i]}, {region}")

#1.0 VS 2.0 Results

In [0]:
%py
query = """
SELECT
    material_group_name,
    region_name,
    material_id
FROM dev.sandbox.pj_ao_reco
WHERE
    recommendation = 'Delist'
    AND material_group_name IN ('PASTA', 'INSTANT NOODLE', 'CUP NOODLE', 'COCONUT OIL', 'OLIVE OIL', 'SUNFLOWER OIL', 'VEGETABLE OIL', 'FRUIT JUICES')
ORDER BY 1, 2, 3
"""
df_v1 = spark.sql(query).toPandas()

query = """
SELECT
    material_group_name,
    region_name,
    EXPLODE(delist_skus_lst) AS material_id
FROM dev.sandbox.pj_ao_v2_results_master
ORDER BY 1, 2, 3
"""
df_v2 = spark.sql(query).toPandas()

merged = df_v1.merge(df_v2, on = ['material_group_name', 'region_name', 'material_id'], how = 'outer', indicator = True)
summary = merged.groupby(['material_group_name', 'region_name'])['_merge'].value_counts().unstack(fill_value = 0).reset_index()
summary = summary.rename(columns = {'both': 'common', 'left_only': 'only_in_1.0', 'right_only': 'only_in_2.0'})
summary

In [0]:
%sql
SELECT
    material_group_name,
    region_name,
    total_skus,
    delist_skus,
    ROUND(sales_perc_delist * 100, 2) AS sales_perc,
    ROUND(profit_perc_delist * 100, 2) AS gp_perc,
    ROUND(current_value) AS current_value,
    ROUND(expected_value) AS expected_value,
    ROUND(predicted_current_margin * 100, 2) AS predicted_current_margin,
    ROUND(predicted_expected_margin * 100, 2) AS predicted_expected_margin,
    ROUND(current_margin * 100, 2) AS current_margin,
    ROUND(expected_margin * 100, 2) AS expected_margin
FROM dev.sandbox.pj_ao_v2_results_master
ORDER BY 1, 2